In [10]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from lxml import html

from PIL import Image
import io
from selenium.common.exceptions import ElementClickInterceptedException

import requests
import time
import csv
import re 
import os

import numpy as np
import pandas as pd
import random

from Alltrails_utils import get_url

In [11]:
#this part of the code below chooses a header at random from a collection of 10 headers
header_list=[{'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36COOKIE: 5=2; ax=v167-7'},
       {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0'},
       {'User-Agent':'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9a1) Gecko/20070308 Minefield/3.0a1'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/79.0.3945.88 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Datanyze; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.24 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Fedora;Linux x86; rv:60.0) Gecko/20100101 Firefox/60.0'}]


def head():
    n = random.random()
    n=int(n*10)
    return header_list[n]

In [12]:
df = get_url()
df.head(5)

https://www.alltrails.com/canada/british-columbia got loaded
getting trails in canada/british-columbia
<html class="location state" dir="ltr" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# alltrails: http://ogp.me/ns/fb/alltrails#" style="">
<script src="https://bam.nr-data.net/1/04959020f1?a=476729&amp;v=1184.ab39b52&amp;to=d1wIFRZXXApQEUoJV1dSEggLVh8VQQIRAA%3D%3D&amp;rst=3012&amp;ck=1&amp;ref=https://www.alltrails.com/canada/british-columbia&amp;qt=2&amp;ap=21&amp;be=305&amp;fe=2524&amp;dc=1473&amp;af=err,xhr,stn,ins,spa&amp;perf=%7B%22timing%22:%7B%22of%22:1602568569840,%22n%22:0,%22u%22:227,%22ue%22:227,%22f%22:7,%22dn%22:7,%22dne%22:7,%22c%22:7,%22ce%22:7,%22rq%22:9,%22rp%22:9,%22rpe%22:13,%22dl%22:248,%22di%22:1463,%22ds%22:1472,%22de%22:2325,%22dc%22:2520,%22l%22:2523,%22le%22:2562%7D,%22navigation%22:%7B%22ty%22:2%7D%7D&amp;fp=535&amp;fcp=535&amp;jsonp=NREUM.setToken" type=

,Title,Rating,Number of ratings,Difficulty,Description,Links


In [13]:
URLs = np.array(df['Links'])

In [14]:
def download_img(img_url, title):
    
    baseDir=os.getcwd() + '\Images\{}'.format(title)
    if not os.path.exists(baseDir):
        os.makedirs(baseDir)
    
    for i, url in enumerate(img_url):
        
        file_name = f"{i}.jpg"

        try:
            image_content = requests.get(url).content

        except Exception as e:
            print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')

            file_path = os.path.join(baseDir, file_name)

            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=85)

            print(f"SAVED - {url} - AT: {file_path}")

        except Exception as e:
            print(f"ERROR - COULD NOT SAVE {url} - {e}")

In [15]:
path_chrome = r"C:\Users\Lenovo\chromedriver"

length = []
elevation = []
route_type = []
Tags = []
Reviews = []

for i in range(5):
    
    url = URLs[i]
    driver = webdriver.Chrome(path_chrome)
    
    driver.get(url)
    time.sleep(2)

    print("{} got loaded    ".format(url))
    
    while(True):
        try:
            driver.find_element_by_xpath('//*[@id="reviews"]/div[3]/button').click()
        except:
            break
    
    soup = bs(driver.page_source, 'html.parser')
    title = soup.find('div', class_="styles-module__content___1GUwP").find('h1')['title']
    print(title + '\n')
    
    desc = soup.find_all('p', class_='xlate-google line-clamp-4')[0].text
    
    details = soup.find_all('span', class_='detail-data')
    length.append(details[0].text.replace(' ', '').split('\n')[1])
    elevation.append(details[1].text.replace(' ', '').split('\n')[1])
    route_type.append(details[2].text)
    
    tags = soup.find_all('span', class_='big rounded active')
    string = ''
    for tag in tags:
        string += (tag.text + ', ')
    
    Tags.append(string)
    
    reviews = soup.find_all('p', itemprop="reviewBody")
    Reviews = [x.text.split('\r')[0] for x in reviews]
    
    photo_url = url.split('?')[0] + '/photos'
    response= requests.get(photo_url,headers=head())
    print(response)
    
    soup = bs(response.content,"html.parser")
    soup_2 = soup.find_all('div', class_='feed-items null')
    soup_3 = soup_2[0].find_all('a', class_='photo-item gallery sm')
    
    img_urls = [img['href'] for img in soup_3 if img['href'] != None][:5]
    download_img(img_urls, title)
    
    driver.close()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
import random
random.randint(1, 5)

4